In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
dataset = pd.DataFrame(pd.read_csv('../input/compressive-strength-of-concrete/compresive_strength_concrete.csv'))

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.isna().sum()

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset[dataset.duplicated() == True]

In [ ]:
dataset.columns

In [ ]:
dataset.rename(columns={"Cement (component 1)(kg in a m^3 mixture)":'Cement',
                               "Blast Furnace Slag (component 2)(kg in a m^3 mixture)":'Blast_Furnace_Slag',
                               "Fly Ash (component 3)(kg in a m^3 mixture)":'Fly_Ash', "Water  (component 4)(kg in a m^3 mixture)": 'Water',
                               "Superplasticizer (component 5)(kg in a m^3 mixture)":'Superplasticizer',
                                "Coarse Aggregate  (component 6)(kg in a m^3 mixture)":'Coarse-Aggregate',
                               "Fine Aggregate (component 7)(kg in a m^3 mixture)": 'Fine_Aggregate',
                                "Age (day)": "Age_day",
                        "Concrete_target ": 'Concrete_target'
                                
                               }, inplace=True )
                         
                   

In [ ]:
dataset.head()

In [ ]:
blast_mean = np.mean(dataset.Blast_Furnace_Slag)
fly_mean = np.mean(dataset.Fly_Ash)
superplastic_mean = np.mean(dataset.Superplasticizer)

In [ ]:
dataset.Blast_Furnace_Slag=dataset.Blast_Furnace_Slag.mask(dataset.Blast_Furnace_Slag==0).fillna(blast_mean)

In [ ]:
dataset.Fly_Ash=dataset.Fly_Ash.mask(dataset.Fly_Ash==0).fillna(fly_mean)

In [ ]:
dataset.Superplasticizer=dataset.Superplasticizer.mask(dataset.Superplasticizer==0).fillna(superplastic_mean)

In [ ]:
dataset

In [ ]:
y = dataset['Concrete compressive strength(MPa, megapascals) ']

In [ ]:
x = dataset.drop(columns='Concrete compressive strength(MPa, megapascals) ')

In [ ]:
x

In [ ]:
# Split dataset for training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
X_train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
# Data Normalization
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train -= mean
X_train /= std

X_test -= mean
X_test /= std


X_val -= mean
X_val /= std

In [ ]:
X_train.head()

In [ ]:
# Let's go
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


In [ ]:
network = Sequential()

In [ ]:
network.add(layers.Dense(6, activation='relu', input_shape=(X_train.shape[1],)))
network.add(layers.Dense(4, activation='relu'))
network.add(layers.Dense(1))

In [ ]:
network.compile(optimizer='adam', loss='mse', metrics='mae')

In [ ]:
history = network.fit(X_train, Y_train, batch_size=30, verbose=0, epochs=200, validation_data=(X_val, Y_val))

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, 201)

In [ ]:
print('Training Loss: ' + str(train_loss[-1]))
print('*'*20)
print('Validation Loss' + str(val_loss[-1]))


In [ ]:
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Val loss')
plt.title('Train and Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
mae_train = history.history['mae']
mae_val= history.history['val_mae']

In [ ]:
print('Training Loss: ' + str(mae_train[-1]))
print('*'*20)
print('Validation Loss' + str(mae_val[-1]))


In [ ]:
plt.plot(epochs, mae_train, label='Training MAE')
plt.plot(epochs, mae_val, label='Val MAE')
plt.title('Train and Val MAE')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()

In [ ]:
prediction = network.predict(X_test)

In [ ]:
network.evaluate(X_test, Y_test)

In [ ]:
prediction.shape

In [ ]:
prediction=prediction.reshape(309)

In [ ]:
output = pd.DataFrame({'Actual': Y_test})

In [ ]:
output['prediction'] = prediction

In [ ]:
output.reset_index(inplace=True)

In [ ]:
output['Absolute Error'] = np.abs(output['prediction'] - output['Actual'])

In [ ]:
output

<center> <h2> Thanks ❤ </h2>